# Extracting data from ttl files

In [1]:
from collections import Counter
from tqdm import tqdm
import numpy as np
import re

In [2]:
with open('data/instance_types_transitive_wkd_uris_en.ttl', 'r', encoding='utf-8') as f:
    text1 = f.read()

In [3]:
triplets_instance = text1.split('\n')[1:-2]

# Obtaining set of identifiers from ontology groups

In [4]:
q_id = set()
r_q = r'dbpedia.org/resource/(.*?)>'

for triplet in triplets_instance:
    if '/Person>' in triplet:
        q_id.add(re.search(r_q,triplet).group(1))
        
q_id = np.array(list(q_id))

In [5]:
del(triplets_instance)
del(text1)

In [6]:
with open('data/mappingbased_literals_wkd_uris_en.ttl', 'r', encoding='utf-8') as f:
    text2 = f.read()

In [7]:
triplets_names = text2.split('\n')[1:-2]

# Creating dictionary (identifier-name)

In [8]:
r_names = r'/name> "([A-Za-z\.\-\' ]*)"@en'
r_q = r'dbpedia.org/resource/(.*?)>'
q2name = dict()
for triplet in tqdm(triplets_names):
    name = re.search(r_names, triplet)
    q_group = re.search(r_q,triplet)
    
    if name is not None:
        q2name[q_group.group(1)] = name.group(1)    


100%|██████████| 10841856/10841856 [00:47<00:00, 225895.59it/s]


In [9]:
del(text2)
del(triplets_names)

# Extracting names of persons using identifiers from ontology groups

In [10]:
person_names = []
for group in tqdm(q_id):
    try:
        person_names.append(q2name[group])
        q2name.pop(group)
    except:
        pass
# Other names are considered non person entities
non_person_names = q2name.values()

100%|██████████| 917509/917509 [00:01<00:00, 536630.23it/s]


In [11]:
print(len(set(person_names)))
Counter(person_names).most_common(10)

645405


[('unknown', 171),
 ('Saint', 51),
 ('Governor', 49),
 ('John Smith', 48),
 ('Blessed', 38),
 ('John Williams', 31),
 ('William Smith', 31),
 ('John Brown', 30),
 ('William', 29),
 ('Venerable', 29)]

In [12]:
print(len(set(non_person_names)))
Counter(non_person_names).most_common(10)

1189451


[('Pennsylvania', 634),
 ("''", 319),
 ('Greatest Hits', 317),
 ('Aliabad', 246),
 ('Hoseynabad', 199),
 ('Grand National', 174),
 ('Hasanabad', 161),
 ('Hajjiabad', 157),
 ('Mohammadabad', 150),
 ('Buddleja davidii', 144)]

# Saving extracted names into text files to preprocess later

In [13]:
def write_txt(file, arr):
    for string in arr:
        file.write(string+'\n')
    print('file saved')

In [14]:
file_person_names = open('persons.txt', 'w', encoding='utf-8')
file_other_names = open('entities.txt','w', encoding='utf-8')

write_txt(file_person_names, person_names)
write_txt(file_other_names, non_person_names)

file_person_names.close()
file_other_names.close()

file saved
file saved


In [15]:
with open('persons.txt', 'r', encoding='utf-8') as f:
    array_per = f.read().split('\n')